In [1]:
2#  !pip install requests pandas tqdm

import os
import re
import time
import json
import csv
import requests
import pandas as pd
from tqdm import tqdm
from typing import List, Dict, Optional

# ---------------------------
# CONFIG - Replace API_KEY
# ---------------------------
API_KEY = "JhnIenwb3246rhRB3pNlw5FR2MuiNx5M4BCQ9DhS"  # <--Semantic Scholar API key
DATA_DIR = "data"
PDF_DIR = os.path.join(DATA_DIR, "pdfs")
METADATA_JSON = os.path.join(DATA_DIR, "metadata.json")
METADATA_CSV = os.path.join(DATA_DIR, "metadata.csv")
SEARCH_LIMIT = 12  # number of results to fetch by default

os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# ---------------------------
# SMART SUGGESTIONS (Feature 1)
# ---------------------------
SUGGESTIONS = {
    "ai": ["artificial intelligence healthcare", "deep learning medical imaging", "explainable AI"],
    "nlp": ["transformers for text summarization", "nlp in healthcare"],
    "ml": ["machine learning for diagnosis", "ml anomaly detection"],
    "covid": ["covid-19 vaccine efficacy", "covid-19 transmission modeling"]
}

def auto_suggest(topic: str):
    key = topic.lower().strip()
    printed = False
    for short, suggestions in SUGGESTIONS.items():
        if key == short:
            print("\n💡 Suggested research keyword variations:")
            for s in suggestions:
                print("   🔸", s)
            printed = True
    if not printed and len(topic.split()) <= 2:
        # show general hints
        print("\n💡 Tip: Consider adding domain or method words (e.g., 'healthcare', 'CNN', 'transformer') for better search results.")
    print()

# ---------------------------
# UTILITIES
# ---------------------------
def sanitize_filename(name: str, max_len: int = 100) -> str:
    name = re.sub(r'[\\/*?:"<>|]', "", name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name[:max_len].strip()

KEYWORD_TAGS = ["disease", "covid", "classification", "deep", "cnn", "transformer",
                "diagnosis", "health", "medical", "image", "nlp", "survey", "review", "anomaly"]

def auto_tags(title: str) -> List[str]:
    title_lower = title.lower()
    tags = [kw for kw in KEYWORD_TAGS if kw in title_lower]
    # also add words longer than 5 chars that appear frequently as naive tag (not too noisy)
    extra = [w for w in set(re.findall(r'\b[a-z]{6,}\b', title_lower)) if w not in tags][:3]
    return list(dict.fromkeys(tags + extra))  # preserve order, uniq

def title_relevance_score(title: str, query: str) -> int:
    # simple scoring: keyword exact matches + token overlap
    title_l = title.lower()
    query_tokens = [t for t in re.findall(r'\w+', query.lower()) if len(t) > 1]
    score = 0
    for t in query_tokens:
        if t in title_l:
            score += 2
        # partial token in title: +1
        for word in re.findall(r'\w+', title_l):
            if t in word and t != word:
                score += 1
    return score

def is_duplicate(title: str, existing_meta: List[Dict]) -> bool:
    t = title.lower().strip()
    for m in existing_meta:
        if m.get("title","").lower().strip() == t:
            return True
    return False

def load_existing_metadata() -> List[Dict]:
    if os.path.exists(METADATA_JSON):
        try:
            with open(METADATA_JSON, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return []
    return []

def save_metadata_list(metadata: List[Dict]):
    with open(METADATA_JSON, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    if metadata:
        keys = metadata[0].keys()
        with open(METADATA_CSV, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=list(keys))
            writer.writeheader()
            writer.writerows(metadata)

# ---------------------------
# SEMANTIC SCHOLAR SEARCH (core)
# ---------------------------
def semantic_scholar_search(query: str, limit: int = SEARCH_LIMIT) -> List[Dict]:
    base = "https://api.semanticscholar.org/graph/v1/paper/search"
    fields = "title,authors,year,url,isOpenAccess,openAccessPdf,externalIds,abstract"
    params = {"query": query, "limit": limit, "fields": fields}
    headers = {"x-api-key": API_KEY}
    resp = requests.get(base, params=params, headers=headers, timeout=30)
    resp.raise_for_status()
    data = resp.json()
    papers = []
    for item in data.get("data", []):
        title = item.get("title", "No title")
        authors = ", ".join([a.get("name","") for a in item.get("authors", [])]) if item.get("authors") else ""
        year = item.get("year", None)
        url = item.get("url", None)
        is_open = bool(item.get("isOpenAccess", False))
        pdf_url = None
        if item.get("openAccessPdf"):
            pdf_url = item["openAccessPdf"].get("url")
        abstract = item.get("abstract", "")
        external = item.get("externalIds", {})
        papers.append({
            "title": title,
            "authors": authors,
            "year": year,
            "url": url,
            "is_open_access": is_open,
            "pdf_url": pdf_url,
            "external_ids": external,
            "abstract": abstract
        })
    return papers

# ---------------------------
# DISPLAY / INSIGHTS (Feature 2 & 3)
# ---------------------------
def prepare_and_rank(papers: List[Dict], query: str, existing_meta: List[Dict]) -> List[Dict]:
    for p in papers:
        p["score"] = title_relevance_score(p["title"], query)
        p["tags"] = auto_tags(p["title"])
        p["is_duplicate"] = is_duplicate(p["title"], existing_meta)
    # sort by score (desc) then year (desc if exists)
    papers_sorted = sorted(papers, key=lambda x: (x["score"], x.get("year") or 0), reverse=True)
    return papers_sorted

def emoji_access(p):
    return "🔓" if p["is_open_access"] else "🔒"

def emoji_pdf(p):
    return "📄" if p.get("pdf_url") else "❌"

def show_insights_table(papers: List[Dict]):
    rows = []
    for i,p in enumerate(papers):
        rows.append({
            "Index": i,
            "Title (short)": (p["title"][:70] + "...") if len(p["title"])>70 else p["title"],
            "Year": p.get("year"),
            "Authors": (p["authors"][:40] + "...") if p.get("authors") and len(p["authors"])>40 else p.get("authors"),
            "Access": emoji_access(p),
            "PDF": emoji_pdf(p),
            "Score": p.get("score", 0),
            "Tags": ", ".join(p.get("tags", [])),
            "Duplicate": "⚠" if p.get("is_duplicate") else ""
        })
    df = pd.DataFrame(rows)
    display(df)
    return df

# ---------------------------
# DOWNLOAD LOGIC
# ---------------------------
def download_file(url: str, dest_path: str, max_retries: int = 3, retry_delay: float = 1.2) -> bool:
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(url, stream=True, timeout=30)
            r.raise_for_status()
            total = int(r.headers.get('content-length', 0))
            with open(dest_path, "wb") as f:
                if total and total > 0:
                    pbar = tqdm(total=total, unit="B", unit_scale=True, desc=f"Downloading", leave=False)
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))
                    pbar.close()
                else:
                    f.write(r.content)
            return True
        except Exception as e:
            print(f"Warning: attempt {attempt} failed for URL: {url[:80]}... Error: {e}")
            time.sleep(retry_delay * attempt)
    return False

def download_selected_papers(papers: List[Dict], selected_indices: List[int], existing_meta: List[Dict]) -> List[Dict]:
    metadata_entries = []
    for idx in selected_indices:
        if idx < 0 or idx >= len(papers):
            print(f"Index {idx} out of range — skipped.")
            continue
        p = papers[idx]
        title_clean = sanitize_filename(p["title"]) or f"paper_{idx}"
        filename = f"{p.get('year') or 'na'}_{title_clean}.pdf"
        dest_path = os.path.join(PDF_DIR, filename)

        entry = {
            "index_selected": idx,
            "title": p["title"],
            "authors": p.get("authors"),
            "year": p.get("year"),
            "source_url": p.get("url"),
            "pdf_url": p.get("pdf_url"),
            "local_path": None,
            "status": None,
            "score": p.get("score"),
            "tags": p.get("tags"),
            "is_duplicate": p.get("is_duplicate")
        }

        if p.get("is_duplicate"):
            entry["status"] = "duplicate_skipped"
            print(f"⚠ Duplicate detected - skipped: {p['title']}")
            metadata_entries.append(entry)
            continue

        if not p.get("pdf_url"):
            entry["status"] = "skipped_no_pdf"
            print(f"⚠ Skipped (no open PDF): {p['title']}")
            metadata_entries.append(entry)
            continue

        print(f"📥 Downloading [{idx}] {p['title']}")
        ok = download_file(p["pdf_url"], dest_path)
        if ok:
            entry["local_path"] = dest_path
            entry["status"] = "downloaded"
            print(f"✅ Saved to {dest_path}")
        else:
            entry["status"] = "failed_download"
            print(f"❌ Failed to download: {p['title']}")
        metadata_entries.append(entry)
    return metadata_entries

# ---------------------------
# MAIN INTERACTIVE FLOW
# ---------------------------
def run_enhanced_flow():
    print("=== Enhanced Milestone-1: Paper Retriever (Your Independent Version) ===\n")
    existing_meta = load_existing_metadata()

    topic = input("Enter your research topic / query: ").strip()
    if not topic:
        print("Topic empty. Exiting.")
        return

    # Feature 1: Smart suggestions
    auto_suggest(topic)

    # Ask year filter optionally (user chose B earlier)
    apply_filter = input("Do you want to filter results by year range? (y/n): ").strip().lower().startswith("y")
    start_year = end_year = None
    if apply_filter:
        try:
            start_year = int(input("Enter start year (e.g., 2019): ").strip())
            end_year = int(input("Enter end year (e.g., 2025): ").strip())
            if start_year > end_year:
                start_year, end_year = end_year, start_year
        except Exception:
            print("Invalid input for years; continuing without year filter.")
            apply_filter = False

    # Perform search
    print("\n🔎 Searching Semantic Scholar...")
    papers = semantic_scholar_search(topic, limit=SEARCH_LIMIT)
    if not papers:
        print("No papers found.")
        return

    # Feature 2 & 4 & 5: ranking, tags, duplicate detection
    papers = prepare_and_rank(papers, topic, existing_meta)

    # optional year filtering
    if apply_filter:
        papers_filtered = [p for p in papers if p.get("year") and start_year <= int(p["year"]) <= end_year]
        print(f"Filtered: {len(papers_filtered)} out of {len(papers)} match year range {start_year}-{end_year}.")
        papers = papers_filtered
        if not papers:
            print("No papers after filtering by year.")
            return

    # show insights table (feature 3)
    print("\n📊 Results insights:")
    show_insights_table(papers)

    # selection
    selection_str = input("Select paper indices to download (comma-separated, e.g. 0,2,3) or 'all' to attempt all: ").strip()
    if selection_str.lower() == "all":
        selected_indices = list(range(len(papers)))
    else:
        try:
            selected_indices = [int(s.strip()) for s in selection_str.split(",") if s.strip()!=""]
        except Exception:
            print("Invalid selection. Exiting.")
            return

    print(f"Selected indices: {selected_indices}")
    if not input("Proceed with downloads? (y/n): ").strip().lower().startswith("y"):
        print("Cancelled.")
        return

    # download selected
    metadata_new = download_selected_papers(papers, selected_indices, existing_meta)

    # combine metadata (append)
    combined_meta = existing_meta + metadata_new
    save_metadata_list(combined_meta)
    print(f"\n📁 Metadata saved to: {METADATA_JSON} and {METADATA_CSV}")
    print(f"📂 PDFs saved under: {PDF_DIR}")

    # show summary table
    if metadata_new:
        display(pd.DataFrame(metadata_new))
    else:
        print("No new metadata generated.")

# Run it:
if __name__ == "__main__":
    run_enhanced_flow()


=== Enhanced Milestone-1: Paper Retriever (Your Independent Version) ===


💡 Tip: Consider adding domain or method words (e.g., 'healthcare', 'CNN', 'transformer') for better search results.


🔎 Searching Semantic Scholar...
Filtered: 5 out of 12 match year range 2020-2025.

📊 Results insights:


,Index,Title (short),Year,Authors,Access,PDF,Score,Tags,Duplicate
0,0,Standards of Care for the Health of Transgende...,2022,"E. Coleman, A. Radix, W. Bouman, G. Brow...",🔓,📄,2,"health, standards, transgender, diverse",
1,1,Health and Human\n Services,2020,"Governor, Kody H. Kinsley, Secretary, Ma...",🔒,❌,2,"health, services",
2,2,Factors Associated With Mental Health Outcomes...,2020,"Jianbo Lai, Simeng Ma, Ying Wang, Zhongx...",🔓,📄,2,"disease, health, factors, mental, coronavirus",
3,3,Health,2020,,🔒,❌,2,health,
4,4,Substance Abuse and Mental Health Services Adm...,2020,"Brian Altman, Jane Powers, Christian Huy...",🔓,📄,2,"health, administration, mental, substance",


Selected indices: [0, 1, 2, 3, 4]
Proceed with downloads? (y/n): y
📥 Downloading [0] Standards of Care for the Health of Transgender and Gender Diverse People, Version 8
❌ Failed to download: Standards of Care for the Health of Transgender and Gender Diverse People, Version 8
⚠ Skipped (no open PDF): Health and Human
 Services
📥 Downloading [2] Factors Associated With Mental Health Outcomes Among Health Care Workers Exposed to Coronavirus Disease 2019
❌ Failed to download: Factors Associated With Mental Health Outcomes Among Health Care Workers Exposed to Coronavirus Disease 2019
⚠ Skipped (no open PDF): Health
📥 Downloading [4] Substance Abuse and Mental Health Services Administration


✅ Saved to data/pdfs/2020_Substance Abuse and Mental Health Services Administration.pdf

📁 Metadata saved to: data/metadata.json and data/metadata.csv
📂 PDFs saved under: data/pdfs


,index_selected,title,authors,year,source_url,pdf_url,local_path,status,score,tags,is_duplicate
0,0,Standards of Care for the Health of Transgende...,"E. Coleman, A. Radix, W. Bouman, G. Brown, A. ...",2022,https://www.semanticscholar.org/paper/a2e5ed3b...,https://doi.org/10.1080/26895269.2022.2100644,None,failed_download,2,"[health, standards, transgender, diverse]",False
1,1,Health and Human\n Services,"Governor, Kody H. Kinsley, Secretary, Mark Payne",2020,https://www.semanticscholar.org/paper/2d501e01...,,None,skipped_no_pdf,2,"[health, services]",False
2,2,Factors Associated With Mental Health Outcomes...,"Jianbo Lai, Simeng Ma, Ying Wang, Zhongxiang C...",2020,https://www.semanticscholar.org/paper/2f946366...,https://jamanetwork.com/journals/jamanetworkop...,None,failed_download,2,"[disease, health, factors, mental, coronavirus]",False
3,3,Health,,2020,https://www.semanticscholar.org/paper/d90e0636...,,None,skipped_no_pdf,2,[health],False
4,4,Substance Abuse and Mental Health Services Adm...,"Brian Altman, Jane Powers, Christian Huygen, C...",2020,https://www.semanticscholar.org/paper/db87118c...,http://files.eric.ed.gov/fulltext/ED445460.pdf,data/pdfs/2020_Substance Abuse and Mental Heal...,downloaded,2,"[health, administration, mental, substance]",False


In [3]:
!pip install pymupdf nltk scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.9 MB/s eta 0:00:00


In [3]:
import os

PDF_DIR = "data/pdfs"

print("Folder exists:", os.path.exists(PDF_DIR))

if os.path.exists(PDF_DIR):
    files = os.listdir(PDF_DIR)
    print("Files inside data/pdfs:", files)
else:
    print("❌ data/pdfs folder not found")


Folder exists: True
Files inside data/pdfs: ['2020_Substance Abuse and Mental Health Services Administration.pdf']


In [4]:
## Extract text
import fitz

files = os.listdir(PDF_DIR)

if not files:
    print("❌ No PDFs found. Upload or download PDFs first.")
else:
    pdf_path = os.path.join(PDF_DIR, files[0])
    print("Using PDF:", pdf_path)

    doc = fitz.open(pdf_path)
    print("Total pages:", len(doc))

    text = ""
    for page in doc:
        text += page.get_text()

    print("\n🔹 STEP 2 OUTPUT (first 500 chars):\n")
    print(text[:500])
    print("\nTotal characters extracted:", len(text))


Using PDF: data/pdfs/2020_Substance Abuse and Mental Health Services Administration.pdf
Total pages: 6

🔹 STEP 2 OUTPUT (first 500 chars):

ED 445 460
AUTHOR
TITLE
INSTITUTION
SPONS AGENCY
PUB DATE
NOTE.
CONTRACT
AVAILABLE FROM
PUB TYPE
EDRS PRICE
DESCRIPTORS
ABSTRACT
DOCUMENT RESUME
EC 308 067
Treder, David; Kutash, Krista; Duchnowski, Albert J.; Rudo,
Zena; Sumi, W. Carl; Harris, Karen M.; Nelson, Steven L.
The School, Family, and Community Partnership Program:
Initial Findings.
University of South Florida, Tampa. Research and Training
Center for Children's Mental Health.; University of South
Florida, Tampa. Louis de la Parte Flor

Total characters extracted: 17825


In [5]:
## Split into sections
import re

def split_into_sections(text):
    patterns = {
        "abstract": r"\babstract\b",
        "introduction": r"\bintroduction\b",
        "methodology": r"\b(method|methodology)\b",
        "results": r"\bresults\b",
        "conclusion": r"\b(conclusion|discussion)\b"
    }

    text_l = text.lower()
    indices = {k: re.search(v, text_l).start()
               for k, v in patterns.items() if re.search(v, text_l)}

    sections = {}
    sorted_items = sorted(indices.items(), key=lambda x: x[1])

    for i, (sec, start) in enumerate(sorted_items):
        end = sorted_items[i+1][1] if i+1 < len(sorted_items) else len(text)
        sections[sec] = text[start:end]

    return sections


sections = split_into_sections(text)

print("\n🔹 STEP 3 OUTPUT – Sections Found:")
print(sections.keys())



🔹 STEP 3 OUTPUT – Sections Found:
dict_keys(['abstract', 'results', 'introduction', 'methodology', 'conclusion'])


In [7]:
# Key findings
import nltk

# Ensure necessary NLTK resources are available
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

# Function to extract key findings (sentence-wise)
def extract_key_findings(text):
    if not text:
        return []
    # Split text into sentences
    sentences = sent_tokenize(text)
    # Here you can add further filtering or keyword extraction if needed
    return sentences

# Example usage with sections dictionary
# sections = {'Introduction': 'Your text here...', 'Results': 'Your results text...'}
for sec, content in sections.items():
    print(f"\n{sec.upper()}:")
    for s in extract_key_findings(content):
        print("•", s)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.



ABSTRACT:
• ABSTRACT
DOCUMENT RESUME
EC 308 067
Treder, David; Kutash, Krista; Duchnowski, Albert J.; Rudo,
Zena; Sumi, W. Carl; Harris, Karen M.; Nelson, Steven L.
The School, Family, and Community Partnership Program:
Initial Findings.
• University of South Florida, Tampa.
• Research and Training
Center for Children's Mental Health.
• ; University of South
Florida, Tampa.
• Louis de la Parte Florida Mental Health
Inst.
• Substance Abuse and Mental Health Services Administration
(DHHS/PHS), Rockville, MD.
• Center for Mental Health
Services.
• ; National Inst.
• on Disability and Rehabilitation
Research (ED/OSERS), Washington, DC.
• 1999-02-00
5p.
• ; In: The Annual Research Conference Proceedings, A
System of Care for Children's Mental Health: Expanding the
Research Base (12th, Tampa, FL, February 21-24, 1999).
• H133B90022
For full text:
http://www.fmhi.usf.edu/institute/pubs/bysubject.html.
• Reports - Research (143)
Speeches/Meeting Papers (150)
MF01/PC01 Plus Postage.
• *Agency 

In [8]:
# Process all PDFs
print("\n🔹 STEP 5 OUTPUT – All PDFs")

for pdf in os.listdir(PDF_DIR):
    path = os.path.join(PDF_DIR, pdf)
    text = extract_text_from_pdf(path)
    sections = split_into_sections(text)

    print("\nPDF:", pdf)
    print("Sections:", list(sections.keys()))




🔹 STEP 5 OUTPUT – All PDFs

PDF: 2020_Substance Abuse and Mental Health Services Administration.pdf
Sections: ['abstract', 'results', 'introduction', 'methodology', 'conclusion']
